https://adventofcode.com/2022/day/11

In [1]:
import numpy as np

Read in puzzle input, or use example input

In [2]:
# puzzle
with open('monkeys.txt', 'r') as f:
    notes = f.read()

That makes sense to a human, but not a program. Let's organize it in a dictionary.

In [3]:
def readnotes(notes):
    monkeybiz = {}
    troop = notes.split('\n\n')
    for monkey in troop:
        attributes = monkey.split('\n')
        name = attributes[0].lower()[:-1]
        monkeybiz[name] = {}
        while len(attributes) > 1:
            attr = attributes.pop(1).strip().split(': ')
            if attr[0] == 'Starting items':
                attr[0] = 'items'
                attr[1] = list(map(int, attr[1].split(', ')))
            if attr[0] == 'Operation':
                attr[0] = 'oper'
                attr[1] = attr[1].replace('new = old ', '')
            if attr[0] == 'Test':
                attr[0] = 'test'
                attr[1] = attr[1].replace('divisible by', '%')
            if attr[0] == 'If true':
                attr[0] = 'true'
                attr[1] = attr[1].replace('throw to ', '')
            if attr[0] == 'If false':
                attr[0] = 'false'
                attr[1] = attr[1].replace('throw to ', '')
            monkeybiz[name][attr[0]] = attr[1]
            monkeybiz[name]['business'] = 0
    return monkeybiz

In [4]:
monkeybiz = readnotes(notes)

monkeybiz

{'monkey 0': {'items': [65, 58, 93, 57, 66],
  'business': 0,
  'oper': '* 7',
  'test': '% 19',
  'true': 'monkey 6',
  'false': 'monkey 4'},
 'monkey 1': {'items': [76, 97, 58, 72, 57, 92, 82],
  'business': 0,
  'oper': '+ 4',
  'test': '% 3',
  'true': 'monkey 7',
  'false': 'monkey 5'},
 'monkey 2': {'items': [90, 89, 96],
  'business': 0,
  'oper': '* 5',
  'test': '% 13',
  'true': 'monkey 5',
  'false': 'monkey 1'},
 'monkey 3': {'items': [72, 63, 72, 99],
  'business': 0,
  'oper': '* old',
  'test': '% 17',
  'true': 'monkey 0',
  'false': 'monkey 4'},
 'monkey 4': {'items': [65],
  'business': 0,
  'oper': '+ 1',
  'test': '% 2',
  'true': 'monkey 6',
  'false': 'monkey 2'},
 'monkey 5': {'items': [97, 71],
  'business': 0,
  'oper': '+ 8',
  'test': '% 11',
  'true': 'monkey 7',
  'false': 'monkey 3'},
 'monkey 6': {'items': [83, 68, 88, 55, 87, 67],
  'business': 0,
  'oper': '+ 2',
  'test': '% 5',
  'true': 'monkey 2',
  'false': 'monkey 1'},
 'monkey 7': {'items': [64, 

Create functions for the inspection and test steps, for clarity.

In [5]:
def inspect(item, oper):
    op = oper.split()[0]
    value = oper.split()[1]
    if value == 'old':
        value = item
    if op == '+':
        item = item + int(value)
    elif op == '*':
        item = item * int(value)
    return item


def test(item, test):
    value = test.split()[1]
    return item % int(value) == 0

All right, let's throw some packages.

In [6]:
rounds = 20

while rounds > 0:
    for monkey in monkeybiz.keys():
        while len(monkeybiz[monkey]['items']) > 0:
            item = int(monkeybiz[monkey]['items'].pop(0))
            item = inspect(item, monkeybiz[monkey]['oper'])
            monkeybiz[monkey]['business'] += 1
            item = int(item/3)
            if test(item, monkeybiz[monkey]['test']):
                catcher = monkeybiz[monkey]['true']
            else:
                catcher = monkeybiz[monkey]['false']
            monkeybiz[catcher]['items'].append(item)
            
    rounds -= 1

In [7]:
for monkey in monkeybiz:
    print (monkey, monkeybiz[monkey]['business'])

monkey 0 51
monkey 1 247
monkey 2 204
monkey 3 201
monkey 4 249
monkey 5 216
monkey 6 53
monkey 7 47


Part 2 took me three days to figure out. 

I tried using gmpy2 package to improve computational speed with large numbers. It helped up to 1000 rounds, but that was the most it could handle in a few minutes. I checked the length of one of Monkey 0's items and the worry level had 48 million digits. 

For the next two days I tried to understand what lower number I could pass that would be processed the same way by all the subsequent monkeys, or whether a small number of paths through the monkeys could be predicted for a particular item.... 

Eventually I noticed that multiplying a number by x had no affect on whether the result is divisible by y, and that for addition if you operated on the remainder instead of the full number the result would be the same: (x + a) % y = (x % y + a) % y. I tried replacing the item with its remainder and passing the result of the inspection along to the next monkey... the computation was fast but the inspection counts were off. Passing the remainder of the test didn't work either.

For two more days I tried to find a way to reduce the item number but still have all the monkeys route the item the same way. I was stuck because when a monkey's inspection alters the value, every monkey's test responds differently. 

Eventually I realized that instead of passing a single number, I would need to pass the remainders for each monkey's test. 

Step 1: express all the items in terms of their remainders.

In [8]:
monkeybiz = readnotes(notes)
divisors = []

for monkey in monkeybiz:
    divisors.append(int(monkeybiz[monkey]['test'].split()[1]))    

divisors = np.array(divisors)

for monkey in monkeybiz:
    monkeybiz[monkey]['remainders'] = []
    for item in monkeybiz[monkey]['items']:        
        r = []
        for i in range(len(divisors)):
            r.append(item % divisors[i])
        r = np.array(r)
        monkeybiz[monkey]['remainders'].append(r)

Modify the inspection, test, and decision slightly to accommodate the new format of the item.

In [9]:
def inspect(item, oper):
    op = oper.split()[0]
    value = oper.split()[1]
    if value == 'old':
        value = item
    else:
        value = int(value)
    if op == '+':
        item = item + value
    elif op == '*':
        item = item * value
    return item

def test(item, divisors):
    return item % divisors

rounds = 10000

while rounds > 0:
    for monkey in monkeybiz.keys():
        monkeynum = int(monkey[-1])
        while len(monkeybiz[monkey]['remainders']) > 0:
            item = monkeybiz[monkey]['remainders'].pop(0)
            item = inspect(item, monkeybiz[monkey]['oper'])
            item = test(item, divisors)
            if item[monkeynum] == 0:
                catcher = monkeybiz[monkey]['true']
            else:
                catcher = monkeybiz[monkey]['false']
            monkeybiz[catcher]['remainders'].append(item)

            monkeybiz[monkey]['business'] += 1
            
    rounds -= 1

In [10]:
for monkey in monkeybiz:
    print (monkey, monkeybiz[monkey]['business'])

monkey 0 45924
monkey 1 131087
monkey 2 87798
monkey 3 95174
monkey 4 107420
monkey 5 96174
monkey 6 74384
monkey 7 41580


In [11]:
131087 * 107420

14081365540